### Importing all the libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

### Creating a Convolutional Neural Network Model

In [2]:
class CNN(nn.Module):
    
    def __init__(self, in_channels, num_classes):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.conv3 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1 = nn.Linear(32*4*4,120)
        self.fc2 = nn.Linear(120,60)
        self.fc3 = nn.Linear(60,num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

### Checking the model on random data

In [3]:
model = CNN(3,10)
x = torch.randn(64,3,32,32)
print(model(x).shape)

torch.Size([64, 10])


In [4]:
arr = model(x)

In [5]:
arr

tensor([[ 4.3238e-02,  1.4621e-01, -1.3712e-02, -4.8171e-02, -1.1338e-01,
         -2.6922e-02,  9.6738e-02,  7.9698e-03, -6.4250e-02,  2.9148e-03],
        [ 4.3549e-02,  1.4207e-01, -1.9574e-02, -5.2452e-02, -1.1164e-01,
         -2.5913e-02,  9.1984e-02,  6.7695e-03, -6.1857e-02,  1.5936e-03],
        [ 4.2473e-02,  1.4674e-01, -2.2778e-02, -5.8635e-02, -1.1206e-01,
         -3.1381e-02,  8.7190e-02,  1.3693e-02, -5.8405e-02,  8.5062e-04],
        [ 4.7629e-02,  1.4723e-01, -2.2115e-02, -5.9518e-02, -1.1543e-01,
         -2.7104e-02,  8.9384e-02,  1.5468e-02, -5.3957e-02,  2.4833e-03],
        [ 4.3611e-02,  1.4145e-01, -1.6574e-02, -5.7222e-02, -1.1190e-01,
         -2.4696e-02,  9.5449e-02,  4.5117e-03, -6.5156e-02,  4.7266e-03],
        [ 3.7957e-02,  1.4873e-01, -2.7225e-02, -5.7269e-02, -1.2076e-01,
         -3.2448e-02,  8.8291e-02,  9.4849e-03, -5.5805e-02,  4.7141e-03],
        [ 4.5041e-02,  1.5079e-01, -1.8506e-02, -6.0789e-02, -1.2472e-01,
         -2.3046e-02,  9.6158e-0

In [6]:
torch.einsum("ij->i",arr)

tensor([ 3.0635e-02,  1.4528e-02,  7.6833e-03,  2.4070e-02,  1.4202e-02,
        -4.3292e-03,  1.4046e-02,  3.6158e-05,  1.2544e-02,  1.5034e-02,
         8.3820e-03,  1.5791e-02,  9.2395e-03,  9.9744e-03,  2.0198e-02,
         3.8573e-03,  1.1539e-02,  4.4544e-03, -1.3200e-02,  2.4588e-03,
         9.4402e-04,  3.4040e-03, -1.6079e-03,  2.9489e-02,  4.9433e-03,
         1.7236e-02,  1.3855e-02,  4.3635e-04,  2.6865e-02,  1.6016e-03,
         3.1328e-02,  1.1350e-04, -3.0588e-03, -5.3433e-04, -1.6801e-03,
         3.7343e-03, -4.8217e-05, -4.4509e-03,  2.9935e-02,  1.0972e-02,
         1.0401e-02,  5.9367e-03,  3.8732e-03,  1.2676e-02,  3.2963e-02,
        -4.5343e-04, -1.5316e-03,  2.3311e-02, -2.0607e-02,  1.0269e-02,
        -1.6997e-02,  7.8399e-04,  2.7856e-03,  3.7171e-02,  1.4927e-02,
        -7.2044e-03, -1.5403e-02,  1.3746e-02,  3.6174e-02,  2.4302e-02,
         1.4814e-02,  9.7466e-03,  2.1569e-02,  2.0031e-02],
       grad_fn=<SumBackward1>)

### Using CUDA if it is available

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Model Parameters

In [8]:
in_channels = 3
num_classes = 10
learning_rate = 0.0001
batch_size = 64
num_epochs = 10

### Data Loading

In [9]:
# CIFAR Dataset
train_dataset = datasets.CIFAR10(root='cifar_data/',train=True, transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.CIFAR10(root='cifar_data/',train=False, transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


### Intialize the Model

In [10]:
model = CNN(in_channels=in_channels,num_classes=num_classes).to(device)

### Loss and Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [12]:
for epoch in tqdm(range(num_epochs)):
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get the data to CUDA if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Descent
        optimizer.step()

100%|███████████████████████████████████████████| 10/10 [02:15<00:00, 13.53s/it]


### Checking the accuracy of the Model

In [13]:
def check_accuracy(loader, model):
    
    if loader.dataset.train:
        print("Checking accuracy on training data")
        
    else :
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    #model.eval()
    
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    
    with torch.no_grad():
        
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
            for i in range(y.size(0)):
                label = y[i]
                pred = predictions[i]
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
            
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        
    #model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy on training data
Accuracy of plane: 57.98 %
Accuracy of car: 68.92 %
Accuracy of bird: 24.6 %
Accuracy of cat: 34.44 %
Accuracy of deer: 31.38 %
Accuracy of dog: 35.34 %
Accuracy of frog: 62.76 %
Accuracy of horse: 56.78 %
Accuracy of ship: 60.62 %
Accuracy of truck: 49.3 %
Got 24106/50000 with accuracy 48.21
Checking accuracy on test data
Accuracy of plane: 60.1 %
Accuracy of car: 70.2 %
Accuracy of bird: 25.0 %
Accuracy of cat: 33.0 %
Accuracy of deer: 30.0 %
Accuracy of dog: 36.9 %
Accuracy of frog: 66.4 %
Accuracy of horse: 58.3 %
Accuracy of ship: 61.7 %
Accuracy of truck: 47.5 %
Got 4891/10000 with accuracy 48.91
